In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

#sklearn test train split
from sklearn.model_selection import train_test_split

# transform data
from sklearn.preprocessing import MinMaxScaler

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


#Imputation and normalization
from sklearn.preprocessing import MinMaxScaler, Imputer

In [14]:
df_1 = pd.read_csv("feature_set_1.csv")
df_2 = pd.read_csv("feature_set_2.csv")
df_3 = pd.read_csv("feature_set_3.csv")
df_1 = df_1.drop(columns=['Unnamed: 0', 'SK_ID_CURR'])
df_2 = df_2.drop(columns=['Unnamed: 0', 'SK_ID_CURR'])
df_3 = df_3.drop(columns=['Unnamed: 0'])

In [17]:
Negatives = df['TARGET'][df['TARGET']==0].count()
Positives = df['TARGET'][df['TARGET']==1].count()
print(Negatives, Positives, "Ratio:", Positives/Negatives)
Pos = df[df['TARGET']==1]

234387 19683 Ratio: 0.08397650040317936


Undersampling Dataset to train

In [27]:
random_samples = df[df.TARGET == 0].sample(n=Positives).index
Neg = df.iloc[random_samples]

In [28]:
under_sampled_set = pd.concat([Pos,Neg])

In [29]:
Y = df['TARGET']
X = df.drop(['TARGET'], axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=0)

Y_sampled = under_sampled_set['TARGET']
X_sampled = under_sampled_set.drop(['TARGET'], axis = 1)

X_train_sampled, X_test_sampled, Y_train_sampled, Y_test_sampled = train_test_split(X_sampled,Y_sampled, test_size=0.3, random_state=0)




In [93]:
df = df.drop(df.index[254069])

In [94]:
#Remove NAs
rem = 0
for i in range(len(df['ORGANIZATION_TYPE_Insurance'])):
    y = df['ORGANIZATION_TYPE_Insurance'].iloc[i]
    if pd.isna(y)!=False:
        print(i)
        rem = i
df = df.drop(df.index[rem])
#Y_test = Y_test.drop(Y_test.index[rem])
#X_test = scaler.transform(X_test)

In [10]:
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, recall_score, precision_score, f1_score

In [30]:
gb = GradientBoostingClassifier(n_estimators=100, random_state = 0)
gb.fit(X_train_sampled, Y_train_sampled)
predictions = gb.predict(X_train_sampled)

print("Confusion Matrix:")
print(confusion_matrix(Y_train_sampled, predictions))
print()
print("Classification Report")
print(classification_report(Y_train_sampled, predictions))

Confusion Matrix:
[[9934 3869]
 [4280 9473]]

Classification Report
             precision    recall  f1-score   support

        0.0       0.70      0.72      0.71     13803
        1.0       0.71      0.69      0.70     13753

avg / total       0.70      0.70      0.70     27556



In [35]:
Y_scores = gb.decision_function(X_test_sampled)
fpr_gb, tpr_gb, _ = roc_curve(Y_test_sampled, Y_scores)
roc_auc_gb = auc(fpr_gb, tpr_gb)

print("Area under ROC curve = {:0.2f}".format(roc_auc_gb))


Area under ROC curve = 0.75


In [36]:
accuracy_score(Y_train_sampled,predictions)

0.7042749310494992

In [37]:
precision_score(Y_train_sampled,predictions)

0.7100134912307

In [38]:
recall_score(Y_train_sampled,predictions)

0.688795171962481

In [39]:
predictions_2 = gb.predict(X_train)

print("Confusion Matrix:")
print(confusion_matrix(Y_train, predictions_2))
print()
print("Classification Report")
print(classification_report(Y_train, predictions_2))

Confusion Matrix:
[[114912  49165]
 [  4357   9415]]

Classification Report
             precision    recall  f1-score   support

        0.0       0.96      0.70      0.81    164077
        1.0       0.16      0.68      0.26     13772

avg / total       0.90      0.70      0.77    177849



In [44]:
accuracy_score(Y_train,predictions_2)

0.6990593143621836

In [45]:
precision_score(Y_train,predictions_2)

0.16072038238306588

In [46]:
recall_score(Y_train,predictions_2)

0.6836334591925646

In [47]:
c = confusion_matrix(Y_train, predictions_2)

In [55]:
TN = c[0][0]
FP = c[0][1]
FN = c[1][0]
TP = c[1][1]

MAKE LOOP FOR DIFFERENT SAMPLING RATIOS

In [15]:
def sampling(data,ratio):
    df = data
    Pos = df[df['TARGET']==1]
    Neg = df[df['TARGET']==0]
    N = round((len(Pos)*ratio))
    random_samples = Neg.sample(n=N).index
    Neg = df.iloc[random_samples]
    sampled_set = pd.concat([Pos,Neg])
    return(sampled_set)
    


In [16]:
def gradient_boosting(data,ratio,test_size):
    df = data
    df_s = sampling(df,ratio)
    Y = df['TARGET']
    X = df.drop(['TARGET'], axis = 1)
    Ys = df_s['TARGET']
    Xs = df_s.drop(['TARGET'], axis = 1)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=test_size, random_state=0)
    Xs_train, Xs_test, Ys_train, Ys_test = train_test_split(Xs,Ys, test_size=test_size, random_state=0)
    gb = GradientBoostingClassifier(n_estimators=100, random_state = 0)
    gb.fit(Xs_train, Ys_train)
    predictions = gb.predict(X_train)
    c = confusion_matrix(Y_train, predictions)
    TN = c[0][0]
    print("True Negatives:",TN)
    FP = c[0][1]
    print("False Positives:",FP)
    FN = c[1][0]
    print("False Negatives:",FN)
    TP = c[1][1]
    print("True Positives:",TP)
    #Y_scores = gb.decision_function(X_test_sampled)
    #fpr_gb, tpr_gb, _ = roc_curve(Y_test_sampled, Y_scores)
    #roc_auc_gb = auc(fpr_gb, tpr_gb)
    acc = accuracy_score(Y_train,predictions)
    precision = precision_score(Y_train,predictions)
    recall = recall_score(Y_train,predictions)
    f1 = f1_score(T_t)
    return(acc,precision,recall,f1)
    

In [163]:
R = [0.5,0.8,1,1.2,1.5,2,4]


for ratio in R:
    print("Ratio: ",ratio)
    a,p,r = gradient_boosting(df,ratio,0.2)
    accs.append(a)
    precisions.append(p)
    recalls.append(r)



Ratio:  4
True Negatives: 187486
False Positives: 78
False Negatives: 15608
True Positives: 82


In [166]:
from bokeh.plotting import figure, output_file, show

output_file("line.html")

p = figure(plot_width=400, plot_height=400)

# add a line renderer
p.line(R[:7], accs[:7], line_width=2, color='blue', legend='accuracy')
p.line(R[:7], precisions[:7], line_width=2, color='green', legend='precision')
p.line(R[:7], recalls[:7], line_width=2, color='red', legend='recall')
p.legend.location = "center_right"
p.xaxis.axis_label = 'Negative to Positive Sampling Ratio'
show(p)

Selecting R = 1. Recall takes priority over everything

In [9]:
from sklearn.model_selection import GridSearchCV, cross_val_score


In [18]:
def scoring(data,gb):
    df = data
    df_s = sampling(df,1)
    Y = df['TARGET']
    X = df.drop(['TARGET'], axis = 1)
    Ys = df_s['TARGET']
    Xs = df_s.drop(['TARGET'], axis = 1)
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=0)
    Xs_train, Xs_test, Ys_train, Ys_test = train_test_split(Xs,Ys, test_size=0.3, random_state=0)
    #gb = GradientBoostingClassifier(n_estimators=100, random_state = 0)
    #Unsampled Training
    gb.fit(X_train, Y_train)
    print('Checkpoint')
    predictions = gb.predict(X_train)
    acc = accuracy_score(Y_train,predictions)
    precision = precision_score(Y_train,predictions)
    recall = recall_score(Y_train,predictions)
    f1 = f1_score(Y_train,predictions)
    print("Unsampled Train accuracy:",acc)
    print("Unsampled Train  precision:",precision)
    print("Unsampled Train recall:",recall)
    print("Unsampled Train f1:",f1)
    #Unsampled Test
    predictions = gb.predict(X_test)
    acc = accuracy_score(Y_test,predictions)
    precision = precision_score(Y_test,predictions)
    recall = recall_score(Y_test,predictions)
    f1 = f1_score(Y_test,predictions)
    print("Unsampled Test accuracy:",acc)
    print("Unsampled Test precision:",precision)
    print("Unsampled Test recall:",recall)
    print("Unsampled Test f1:",f1)
    #Sampled Training
    gb.fit(Xs_train, Ys_train)
    predictions = gb.predict(X_train)
    acc = accuracy_score(Y_train,predictions)
    precision = precision_score(Y_train,predictions)
    recall = recall_score(Y_train,predictions)
    f1 = f1_score(Y_train,predictions)
    print("Sampled Train accuracy:",acc)
    print("Sampled Train precision:",precision)
    print("Sampled Train recall:",recall)
    print("Sampled Train f1:",f1)
    #Sampled Test
    predictions = gb.predict(X_test)
    acc = accuracy_score(Y_test,predictions)
    precision = precision_score(Y_test,predictions)
    recall = recall_score(Y_test,predictions)
    f1 = f1_score(Y_test,predictions)
    print("Sampled Test accuracy:",acc)
    print("Sampled Test precision:",precision)
    print("Sampled Test recall:",recall)
    print("Sampled Test f1:",f1)
    return

In [19]:
gb = GradientBoostingClassifier(random_state = 0,n_estimators=150, max_depth = 5)
scoring(df_1,gb)

Checkpoint
Unsampled Train accuracy: 0.9218654910177138
Unsampled Train  precision: 0.8153409090909091
Unsampled Train recall: 0.04924220760651987
Unsampled Train f1: 0.09287524944717114
Unsampled Test accuracy: 0.9201335443449606
Unsampled Test precision: 0.4606741573033708
Unsampled Test recall: 0.0223433242506812
Unsampled Test f1: 0.04261954261954262
Sampled Train accuracy: 0.6932364568864194
Sampled Train precision: 0.17058408424709587
Sampled Train recall: 0.7189019159279383
Sampled Train f1: 0.27573952814978114
Sampled Test accuracy: 0.6944414334337806
Sampled Test precision: 0.16947271631947747
Sampled Test recall: 0.7282016348773842
Sampled Test f1: 0.2749556315748862


In [20]:
gb = GradientBoostingClassifier(random_state = 0,n_estimators=150, max_depth = 5)
scoring(df_2,gb)

Checkpoint
Unsampled Train accuracy: 0.9222975327167061
Unsampled Train  precision: 0.8868501529051988
Unsampled Train recall: 0.04975693451529883
Unsampled Train f1: 0.09422722841979855
Unsampled Test accuracy: 0.9201769028985193
Unsampled Test precision: 0.4642857142857143
Unsampled Test recall: 0.02125340599455041
Unsampled Test f1: 0.04064616988014591
Sampled Train accuracy: 0.693845031752742
Sampled Train precision: 0.17178920538510556
Sampled Train recall: 0.7246782956820131
Sampled Train f1: 0.27773881022313796
Sampled Test accuracy: 0.6949942549916535
Sampled Test precision: 0.17039619651347068
Sampled Test recall: 0.732425068119891
Sampled Test f1: 0.2764721007971201


In [21]:
gb = GradientBoostingClassifier(random_state = 0,n_estimators=150, max_depth = 5)
scoring(df_3,gb)

Checkpoint
Unsampled Train accuracy: 0.9228782339250291
Unsampled Train  precision: 0.827893175074184
Unsampled Train recall: 0.06382613668859022
Unsampled Train f1: 0.11851537195348591
Unsampled Test accuracy: 0.9200034686842847
Unsampled Test precision: 0.4574468085106383
Unsampled Test recall: 0.029291553133514985
Unsampled Test f1: 0.05505761843790013
Sampled Train accuracy: 0.7004464430889588
Sampled Train precision: 0.1764965031114048
Sampled Train recall: 0.7331998856162425
Sampled Train f1: 0.2845063858590118
Sampled Test accuracy: 0.7009452164675787
Sampled Test precision: 0.1747357599511678
Sampled Test recall: 0.741008174386921
Sampled Test f1: 0.28278784412613406
